In [445]:
import numpy as np
from operator import attrgetter
import string
import random

In [545]:
class Individual(object):
    
    def __init__(self, gene=None, mutation_rate=0.01):
        self.mutation_rate = mutation_rate
        self.letters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz!?.,;- '
        self.gene = gene
        self.score = 0
        if gene == None:
            self.gene_length = 0
        else:
            self.gene_length = len(self.gene)
        
    def generate(self, gene_length):
        self.gene = [random.choice(self.letters) for _ in range(gene_length)]
        self.gene_length = len(self.gene)
        
    def fitness(self, target):
        assert len(target) == len(self.gene)
        self.score = 0
        index = 0
        while index < len(target):
            if self.gene[index] == target[index]:
                self.score += 1
            index += 1
        self.score /= len(target)
        
    def cross_over(self, parent):
        cross_over_point = len(self.gene) // 2
        child = []
        for i in range(0, cross_over_point):
            child.append(self.gene[i])
        for i in range(cross_over_point, self.gene_length):
            child.append(parent.gene[i])
        return Individual(gene=child, mutation_rate=self.mutation_rate)
        
    def mutate(self):
        for i in range(self.gene_length):
            prob = np.random.random_sample()
            if prob > self.mutation_rate:
                continue
            self.gene[i] = random.choice(self.letters)
    

In [570]:
class Population(object):
    
    def __init__(self, target, 
                 initial_population=200, 
                 max_generations=1000,
                 mutation_rate=0.01, 
                 elitism=True,
                 tournament_size=4,
                 solution_threshold=1.0):
        self.target = target
        self.target_length = len(target)
        self.initial_population = initial_population
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.elitism = elitism
        self.tournament_size = tournament_size
        self.solution_threshold = solution_threshold
        self.best_individual = None
        self.best_solution = 0.0
        self.generation = 0
        self.population = []
        
    def _evaluate_fitnesses_(self):
        for p in self.population:
            p.fitness(target)
        self.best_solution = self.evaluate_best_individual().score
        self.best_individual = self.evaluate_best_individual()

    def _perform_tournament_selection_(self):
        tournament = Population(target)
        for _ in range(self.tournament_size):
            i = random.choice(self.population)
            tournament.add_individual(i)
        return tournament.evaluate_best_individual()
    
    def generate(self):
        assert self.initial_population % 2 == 0
        self.population = [Individual(mutation_rate=self.mutation_rate) for _ in range(self.initial_population)]
        for p in self.population:
            p.generate(self.target_length)
    
    def evolve_generation(self):
        children_genes = []
        self._evaluate_fitnesses_()  
        
        if self.elitism:
            children_genes.append(self.best_individual)
            
        for _ in range(self.initial_population):
            # perform selection
            male = self._perform_tournament_selection_()
            female = self._perform_tournament_selection_()
            # perform cross-over
            new_individual = male.cross_over(female)
            # perform mutation
            new_individual.mutate()
            children_genes.append(new_individual)
            
        self.population = children_genes
        self.generation += 1
            
    def evaluate_best_individual(self):
        return max(self.population, key=attrgetter('score'))
    
    def found_solution(self):
        return self.best_solution >= self.solution_threshold
    
    def add_individual(self, individual):
        self.population.append(individual)
    
    def print_score(self):
        print('Generation: {}/{}, Best solution score: {:1.2%}, Text: {}'.format(
            self.generation, 
            self.max_generations,
            self.best_solution,
            ''.join(self.evaluate_best_individual().gene)), end='\r')    

In [571]:
target = 'This is a genetic algorithm test phrase and it works quite well!'

# hypter-parameters
max_generations = 1000
population_size = 200
mutation_rate = 0.01

In [572]:
def execute(target, max_generations, population_size, mutation_rate, display_interval=10):
    population = Population(target, population_size, max_generations)
    population.generate()
    while not population.found_solution() and population.generation < max_generations:
        population.evolve_generation()
        if population.generation % display_interval == 0:
            population.print_score()
    best_individual = population.best_individual
    print('\nGenerations: {}, Final result: {}'.format(population.generation, ''.join(best_individual.gene)))

In [573]:
execute(target, max_generations, population_size, mutation_rate)

Generation: 450/1000, Best solution score: 98.44%, Text: This is a genetic algorithm test phrase and it works quiie well!
Generations: 452, Final result: This is a genetic algorithm test phrase and it works quite well!
